In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
(mnist_train, mnist_test), ds_info = tfds.load('mnist', split=['train', 'test'], as_supervised=True, with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...:   0%|          | 0/4 [00:00<?, ? url/s]
/home/anubhav/anaconda3/envs/genomics/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'storage.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/anubhav/anaconda3/envs/genomics/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'storage.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/anubhav/anaconda3/envs/genomics/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'storage.googleapis.com'. Adding certificate v

0 examples [00:00, ? examples/s]2022-03-22 11:46:32.920641: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-22 11:46:32.935162: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-22 11:46:32.972087: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
Shuffling...:   0%|          | 0/10 [00:00<?, ? shard/s]

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset mnist downloaded and prepared to /home/anubhav/tensorflow_datasets/mnist/1.0.0. Subsequent calls will reuse this data.


In [3]:
def normalize_img(image, label):
  return tf.cast(image, tf.float32) / 255., label

def transform_labels(image, label):
  return image, tf.math.floor(label / 2)

def prepare(ds, shuffle=True, batch_size=32, prefetch=True):
  ds = ds.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
  ds = ds.map(transform_labels, num_parallel_calls=tf.data.experimental.AUTOTUNE)
  ds = ds.shuffle(ds_info.splits['train'].num_examples) if shuffle else ds
  ds = ds.cache()
  ds = ds.batch(batch_size)
  ds = ds.prefetch(tf.data.experimental.AUTOTUNE) if prefetch else ds
  return ds

def split_tasks(ds, predicate):
  return ds.filter(predicate), ds.filter(lambda img, label: not predicate(img, label))

multi_task_train, multi_task_test = prepare(mnist_train), prepare(mnist_test)
task_A_train, task_B_train = split_tasks(mnist_train, lambda img, label: label % 2 == 0)
task_A_train, task_B_train = prepare(task_A_train), prepare(task_B_train)
task_A_test, task_B_test = split_tasks(mnist_test, lambda img, label: label % 2 == 0)
task_A_test, task_B_test = prepare(task_A_test), prepare(task_B_test)

In [4]:
def evaluate(model, test_set):
  acc = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')
  for i, (imgs, labels) in enumerate(test_set):
    preds = model.predict_on_batch(imgs)
    acc.update_state(labels, preds)
  return acc.result().numpy()

In [5]:
multi_task_model = tf.keras.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(5)
])

multi_task_model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics='accuracy')

multi_task_model.fit(multi_task_train, epochs=6)

Epoch 1/6


2022-03-22 11:47:22.820993: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-03-22 11:47:22.860373: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2899885000 Hz


1875/1875 [==============================] - 18s 8ms/step - loss: 0.3899 - accuracy: 0.8750
Epoch 2/6
1875/1875 [==============================] - 9s 5ms/step - loss: 0.1146 - accuracy: 0.9639
Epoch 3/6
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0773 - accuracy: 0.9756
Epoch 4/6
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0569 - accuracy: 0.9825
Epoch 5/6
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0429 - accuracy: 0.9872
Epoch 6/6
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0328 - accuracy: 0.9908


In [6]:
print("Task A accuracy after training on Multi-Task Problem: {}".format(evaluate(multi_task_model, task_A_test)))
print("Task B accuracy after training on Multi-Task Problem: {}".format(evaluate(multi_task_model, task_B_test)))

Task A accuracy after training on Multi-Task Problem: 0.9756394624710083
Task B accuracy after training on Multi-Task Problem: 0.9728025197982788


In [7]:
basic_cl_model = tf.keras.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(5)
])

basic_cl_model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics='accuracy')

basic_cl_model.fit(task_A_train, epochs=6)

Epoch 1/6
922/922 [==============================] - 9s 8ms/step - loss: 0.2544 - accuracy: 0.9245
Epoch 2/6
922/922 [==============================] - 6s 7ms/step - loss: 0.0693 - accuracy: 0.9800
Epoch 3/6
922/922 [==============================] - 7s 7ms/step - loss: 0.0453 - accuracy: 0.9872
Epoch 4/6
922/922 [==============================] - 6s 6ms/step - loss: 0.0312 - accuracy: 0.9913
Epoch 5/6
922/922 [==============================] - 6s 7ms/step - loss: 0.0215 - accuracy: 0.9944
Epoch 6/6
922/922 [==============================] - 7s 7ms/step - loss: 0.0145 - accuracy: 0.9964


In [8]:
print("Task A accuracy after training model on only Task A: {}".format(evaluate(basic_cl_model, task_A_test)))

Task A accuracy after training model on only Task A: 0.9872106909751892


In [9]:
basic_cl_model.fit(task_B_train, epochs=6)

Epoch 1/6
954/954 [==============================] - 11s 9ms/step - loss: 0.1939 - accuracy: 0.9489
Epoch 2/6
954/954 [==============================] - 6s 6ms/step - loss: 0.0618 - accuracy: 0.9815
Epoch 3/6
954/954 [==============================] - 6s 6ms/step - loss: 0.0418 - accuracy: 0.9875
Epoch 4/6
954/954 [==============================] - 7s 8ms/step - loss: 0.0294 - accuracy: 0.9920
Epoch 5/6
954/954 [==============================] - 6s 6ms/step - loss: 0.0205 - accuracy: 0.9944
Epoch 6/6
954/954 [==============================] - 6s 6ms/step - loss: 0.0141 - accuracy: 0.9968


In [10]:
print("Task B accuracy after training trained model on Task B: {}".format(evaluate(basic_cl_model, task_B_test)))
print("Task A accuracy after training trained model on Task B: {}".format(evaluate(basic_cl_model, task_A_test)))

Task B accuracy after training trained model on Task B: 0.9856129288673401
Task A accuracy after training trained model on Task B: 0.27994316816329956


In [12]:
def l2_penalty(theta, theta_A):
  penalty = 0
  for i, theta_i in enumerate(theta):
    _penalty = tf.math.reduce_sum((theta_i - theta_A[i]) ** 2)
    penalty += _penalty
  return 0.5*penalty

def train_with_l2(model, task_A_train, task_B_train, task_A_test, task_B_test, epochs=6):
  # First we're going to fit to task A and retain a copy of parameters trained on Task A
  model.fit(task_A_train, epochs=epochs)
  theta_A = {n: p.value() for n, p in enumerate(model.trainable_variables.copy())}

  print("Task A accuracy after training on Task A: {}".format(evaluate(model, task_A_test)))

  accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
  loss = tf.keras.metrics.SparseCategoricalCrossentropy('loss')

  for epoch in range(epochs):
    accuracy.reset_states()
    loss.reset_states()
    for batch, (imgs, labels) in enumerate(task_B_train):
      with tf.GradientTape() as tape:
        preds = model(imgs)
        total_loss = model.loss(labels, preds) + l2_penalty(model.trainable_variables, theta_A)
      grads = tape.gradient(total_loss, model.trainable_variables)
      model.optimizer.apply_gradients(zip(grads, model.trainable_variables))
      
      accuracy.update_state(labels, preds)
      loss.update_state(labels, preds)
      print("\rEpoch: {}, Batch: {}, Loss: {:.3f}, Accuracy: {:.3f}".format(
          epoch+1, batch+1, loss.result().numpy(), accuracy.result().numpy()), flush=True, end=''
         )
    print("")
  
  print("Task B accuracy after training trained model on Task B: {}".format(evaluate(model, task_B_test)))
  print("Task A accuracy after training trained model on Task B: {}".format(evaluate(model, task_A_test)))

In [13]:
l2_model = tf.keras.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(5)
])

l2_model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics='accuracy')

train_with_l2(l2_model, task_A_train, task_B_train, task_A_test, task_B_test)

Epoch 1/6
922/922 [==============================] - 6s 6ms/step - loss: 0.2549 - accuracy: 0.9235
Epoch 2/6
922/922 [==============================] - 9s 10ms/step - loss: 0.0697 - accuracy: 0.9797
Epoch 3/6
922/922 [==============================] - 6s 7ms/step - loss: 0.0444 - accuracy: 0.9873
Epoch 4/6
922/922 [==============================] - 7s 7ms/step - loss: 0.0304 - accuracy: 0.9917
Epoch 5/6
922/922 [==============================] - 4s 4ms/step - loss: 0.0206 - accuracy: 0.9947
Epoch 6/6
922/922 [==============================] - 5s 5ms/step - loss: 0.0138 - accuracy: 0.9967
Task A accuracy after training on Task A: 0.9863986968994141
Epoch: 1, Batch: 954, Loss: 4.718, Accuracy: 0.554
Epoch: 2, Batch: 954, Loss: 4.611, Accuracy: 0.563
Epoch: 3, Batch: 954, Loss: 4.600, Accuracy: 0.563
Epoch: 4, Batch: 954, Loss: 4.600, Accuracy: 0.563
Epoch: 5, Batch: 954, Loss: 4.601, Accuracy: 0.564
Epoch: 6, Batch: 954, Loss: 4.600, Accuracy: 0.563
Task B accuracy after training trained

In [16]:
def compute_precision_matrices(model, task_set, num_batches=1, batch_size=32):
  task_set = task_set.repeat()
  precision_matrices = {n: tf.zeros_like(p.value()) for n, p in enumerate(model.trainable_variables)}

  for i, (imgs, labels) in enumerate(task_set.take(num_batches)):
    # We need gradients of model params
    with tf.GradientTape() as tape:
      # Get model predictions for each image
      preds = model(imgs)
      # Get the log likelihoods of the predictions
      ll = tf.nn.log_softmax(preds)
    # Attach gradients of ll to ll_grads
    ll_grads = tape.gradient(ll, model.trainable_variables)
    # Compute F_i as mean of gradients squared
    for i, g in enumerate(ll_grads):
      precision_matrices[i] += tf.math.reduce_mean(g ** 2, axis=0) / num_batches

  return precision_matrices

def compute_elastic_penalty(F, theta, theta_A, alpha=25):
  penalty = 0
  for i, theta_i in enumerate(theta):
    _penalty = tf.math.reduce_sum(F[i] * (theta_i - theta_A[i]) ** 2)
    penalty += _penalty
  return 0.5*alpha*penalty

def ewc_loss(labels, preds, model, F, theta_A):
  loss_b = model.loss(labels, preds)
  penalty = compute_elastic_penalty(F, model.trainable_variables, theta_A)
  return loss_b + penalty

def train_with_ewc(model, task_A_set, task_B_set, task_A_test, task_B_test, epochs=3):
  # First we're going to fit to task A and retain a copy of parameters trained on Task A
  model.fit(task_A_set, epochs=epochs)
  theta_A = {n: p.value() for n, p in enumerate(model.trainable_variables.copy())}
  # We'll only compute Fisher once, you can do it whenever
  F = compute_precision_matrices(model, task_A_set, num_batches=1000)

  print("Task A accuracy after training on Task A: {}".format(evaluate(model, task_A_test)))

  # Now we set up the training loop for task B with EWC
  accuracy = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
  loss = tf.keras.metrics.SparseCategoricalCrossentropy('loss')

  for epoch in range(epochs*3):
    accuracy.reset_states()
    loss.reset_states()

    for batch, (imgs, labels) in enumerate(task_B_set):
      with tf.GradientTape() as tape:
        # Make the predictions
        preds = model(imgs)
        # Compute EWC loss
        total_loss = ewc_loss(labels, preds, model, F, theta_A)
      # Compute the gradients of model's trainable parameters wrt total loss
      grads = tape.gradient(total_loss, model.trainable_variables)
      # Update the model with gradients
      model.optimizer.apply_gradients(zip(grads, model.trainable_variables))
      # Report updated loss and accuracy
      accuracy.update_state(labels, preds)
      loss.update_state(labels, preds)
      print("\rEpoch: {}, Batch: {}, Loss: {:.3f}, Accuracy: {:.3f}".format(
          epoch+1, batch+1, loss.result().numpy(), accuracy.result().numpy()), flush=True, end=''
         )
    print("")

  print("Task B accuracy after training trained model on Task B: {}".format(evaluate(model, task_B_test)))
  print("Task A accuracy after training trained model on Task B: {}".format(evaluate(model, task_A_test)))

In [17]:
ewc_model = tf.keras.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(5)
])

ewc_model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics='accuracy')

train_with_ewc(ewc_model, task_A_train, task_B_train, task_A_test, task_B_test)

Epoch 1/3
922/922 [==============================] - 6s 6ms/step - loss: 0.3455 - accuracy: 0.8861
Epoch 2/3
922/922 [==============================] - 4s 5ms/step - loss: 0.1111 - accuracy: 0.9684
Epoch 3/3
922/922 [==============================] - 6s 6ms/step - loss: 0.0853 - accuracy: 0.9747
Task A accuracy after training on Task A: 0.9815266132354736
Epoch: 1, Batch: 954, Loss: 9.046, Accuracy: 0.2267
Epoch: 2, Batch: 954, Loss: 8.184, Accuracy: 0.282
Epoch: 3, Batch: 954, Loss: 8.098, Accuracy: 0.287
Epoch: 4, Batch: 954, Loss: 8.074, Accuracy: 0.289
Epoch: 5, Batch: 954, Loss: 8.062, Accuracy: 0.291
Epoch: 6, Batch: 954, Loss: 8.058, Accuracy: 0.292
Epoch: 7, Batch: 954, Loss: 8.057, Accuracy: 0.293
Epoch: 8, Batch: 954, Loss: 8.062, Accuracy: 0.293
Epoch: 9, Batch: 954, Loss: 8.066, Accuracy: 0.293
Task B accuracy after training trained model on Task B: 0.30153724551200867
Task A accuracy after training trained model on Task B: 0.9236702919006348
